# 1. LOAD DeepSeek

In [1]:
# ref: https://gist.github.com/vgel/8a2497dc45b1ded33287fa7bb6cc1adc
from transformers import AutoModelForCausalLM, AutoTokenizer, DynamicCache
import torch

device=torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model_name='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B'
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained('/scratch/ss4yd/huggingfacemodels/DeepSeek-R1-Distill-Qwen-7B/', 
                                             torch_dtype=torch.bfloat16, device_map='auto')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# 2. Generate Prompt from Opinions QA dataset

In [7]:
from process_opinions_qa import generate_prompt
import random
import yaml

# read yaml file
# change survey_type and context in config.yaml file for different types of prompts as described in the paper
with open('config.yaml') as file:
    config = yaml.safe_load(file)
print(f"Context: {config['prompt']['context']} \nSurvey file: {config['prompt']['survey_type']}")

manual=True # use config file if False, set manually if True
if manual:
    config=dict()
    config['prompt']=dict()
    config['prompt']['context']='steer-qa' # values: default, steer-qa, steer-bio, steer-portray
    config['prompt']['survey_type']='Pew_American_Trends_Panel_disagreement_500' # values: Pew_American_Trends_Panel_W26, replace 26 with the [27,27,29,32..] etc.
    config['prompt']['output_path']='./'
    config['prompt']['include_output']=False
    config['prompt']['reference_index']=None
    
train, evals=generate_prompt(config)

# print(train[0])

if len(train)>0:
    prompt=train[random.choice(range(len(train)))]+'\n'+evals[random.choice(range(len(evals)))]
else:
    prompt=evals[random.choice(range(len(evals)))]

print('\nGenerated prompt:')
print(prompt)

Context: steer-bio 
Survey file: Pew_American_Trends_Panel_disagreement_500

Generated prompt:
Question: What sex were you assigned at birth?
A. Male
B. Female
C. Refused
Answer: A

Question: In general, would you say your view of environmental research scientists is
A. Mostly positive
B. Mostly negative
C. Neither positive nor negative
D. Refused
Answer:


# 3. Prompt model with chain-of-thought prompting

Taken from: https://gist.github.com/vgel/8a2497dc45b1ded33287fa7bb6cc1adc

Code forces the model to generate at least 128 tokens before stopping. The code also used replacement tokens to bridge this.

In [8]:
# ref: https://gist.github.com/vgel/8a2497dc45b1ded33287fa7bb6cc1adc

_, _start_think_token, end_think_token = tokenizer.encode("<think></think>")

replacements=["\nWait, but", "\nHmm", "\nSo"]
@torch.inference_mode
def reasoning_effort(question: str, min_thinking_tokens: int):
    tokens = tokenizer.apply_chat_template(
        [
            {"role": "user", "content": question},
            {"role": "assistant", "content": "<think>\n" + ""},
        ],
        continue_final_message=True,
        return_tensors="pt",
    )
    tokens = tokens.to(model.device)
    kv = DynamicCache()
    n_thinking_tokens = 0

    print(tokenizer.decode(list(tokens[0])))
    while True:
        out = model(input_ids=tokens, past_key_values=kv, use_cache=True)
        next_token = torch.multinomial(
            torch.softmax(out.logits[0, -1, :], dim=-1), 1
        ).item()
        kv = out.past_key_values

        if (
            next_token in (end_think_token, model.config.eos_token_id)
            and n_thinking_tokens < min_thinking_tokens
        ):
            replacement = random.choice(replacements)
            print(replacement)
            replacement_tokens = tokenizer.encode(replacement)
            n_thinking_tokens += len(replacement_tokens)
            tokens = torch.tensor([replacement_tokens]).to(tokens.device)
        elif next_token == model.config.eos_token_id:
            break
        else:
            yield tokenizer.decode([next_token])
            n_thinking_tokens += 1
            tokens = torch.tensor([[next_token]]).to(tokens.device)

## 3a. Run model with CoT prompting

In [9]:
for chunk in reasoning_effort(prompt, 128):
    print(chunk, end="", flush=True)

<｜begin▁of▁sentence｜><｜User｜>Question: What sex were you assigned at birth?
A. Male
B. Female
C. Refused
Answer: A

Question: In general, would you say your view of environmental research scientists is
A. Mostly positive
B. Mostly negative
C. Neither positive nor negative
D. Refused
Answer:<｜Assistant｜><think>

Okay, so the user provided a question about environmental research scientists' general view and asked for an answer. Looking at the previous interaction, the assistant chose option C, "Neither positive nor negative." 

Hmm, the user has a history of asking about gender and then a question about a specific group's perception. This time, it's about environmental research scientists. The options are mostly positive, mostly negative, neither, or refused.

I should consider what factors influence someone's view of environmental scientists. They might value their work since it contributes to sustainability and mitigating climate change. Many people appreciate their efforts in combatin

# 3b Run model without CoT prompting

This code was adapted from https://github.com/stanford-crfm/helm cited in the opinions-qa github repo to reproduce their results. It forces the model to generate only one token, which is usually one of the options presented in the prompt.

In [11]:
stopping_criteria = None
raw_request={
    "prompt":prompt,
    "stop_sequences": [],
    "temperature":1e-7,
    "max_new_tokens":1,
    "top_p":1,
    "num_return_sequences":1,
    "echo_prompt":False
}

encoded_input = tokenizer(raw_request["prompt"], return_tensors="pt", return_token_type_ids=False).to(device)

output = model.generate(
                **encoded_input,
                temperature=raw_request["temperature"],
#                 temperature=None,
                num_return_sequences=raw_request["num_return_sequences"],
                max_new_tokens=raw_request["max_new_tokens"],
                top_p=raw_request["top_p"],
#                 do_sample=False,
                do_sample=True,
                return_dict_in_generate=True,
                output_scores=True,
                output_logits=True,
#                 **optional_args,
                stopping_criteria=stopping_criteria,
            )
sequences = output.sequences
scores = output.logits

print(tokenizer.decode(sequences[0]))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


<｜begin▁of▁sentence｜>Question: What sex were you assigned at birth?
A. Male
B. Female
C. Refused
Answer: A

Question: In general, would you say your view of environmental research scientists is
A. Mostly positive
B. Mostly negative
C. Neither positive nor negative
D. Refused
Answer: A
